In [9]:
from flask import Flask, jsonify
import bl
from flask_limiter import Limiter
from flask_limiter.util import get_remote_address
import waitress
import os


RATE_LIMIT_DAILY = os.getenv('RATE_LIMIT_DAILY')
RATE_LIMIT_HOURLY = os.getenv('RATE_LIMIT_HOURLY')

if RATE_LIMIT_DAILY is None:
    RATE_LIMIT_DAILY = 20

if RATE_LIMIT_HOURLY is None:
    RATE_LIMIT_HOURLY = 5


app = Flask(__name__)
limiter = Limiter(
    app,
    key_func=get_remote_address,
    default_limits=["{} per day".format(
        RATE_LIMIT_DAILY), "{} per hour".format(RATE_LIMIT_HOURLY)]
)


@app.route('/station/<string:id>')
def get_station(id):
    if id == 'all':
        result = bl.get_all_stations()
    else:
        result = bl.get_station_by_id(int(id))

    return jsonify(result), result['code']


@app.route('/weather/<int:id>')
def get_station_weather(id):
    result = bl.get_station_weather(id)
    return jsonify(result), result['code']


if __name__ == '__main__':
    # Uncomment the waitress line and
    # comment the flask (app.run) line
    # on a production environment,
    # or vice versa.

    waitress.serve(app, host='0.0.0.0', port=1875)
    # app.run(host='0.0.0.0', port=1875)

TypeError: __init__() got multiple values for argument 'key_func'

In [10]:
pip install waitress

Note: you may need to restart the kernel to use updated packages.


In [6]:
import json
import scraper


def get_all_stations():
    file = open('stations.json')
    file_data = json.load(file)
    file.close()
    return {
        'code': 200,
        'result': file_data
    }


def get_station_by_id(id):
    file = open('stations.json')
    file_data = json.load(file)
    file.close()
    for station in file_data:
        if station['stationId'] == id:
            return {
                'code': 200,
                'result': station
            }

    return {
        'code': 404,
        'msg': 'No station with ID {} found'.format(id)
    }


def get_station_weather(id):
    data = scraper.get_station_data(id)
    return {
        'code': 200,
        'result': data
    }


In [7]:
import requests
from bs4 import BeautifulSoup


def get_station_data(id):
    URL = 'http://city.imd.gov.in/citywx/city_weather.php?id={}'.format(id)

    response = requests.get(URL, verify=False)
    html_text = response.text

    soup = BeautifulSoup(html_text, 'html.parser')

    cells = soup.find_all('td')

    max_temp = cells[4].text.strip()
    max_dep = cells[6].text.strip()
    min_temp = cells[8].text.strip()
    min_dep = cells[10].text.strip()
    rh_0830 = cells[14].text.strip()
    rh_1730 = cells[16].text.strip()

    sunrise = cells[20].text.strip()
    sunset = cells[18].text.strip()
    moonrise = cells[24].text.strip()
    moonset = cells[22].text.strip()

    day1_date = cells[31].font.text.strip()
    day1_max = cells[33].font.text.strip()
    day1_min = cells[32].font.text.strip()
    day1_forecast = cells[35].font.text.strip()

    day2_date = cells[36].font.text.strip()
    day2_max = cells[38].font.text.strip()
    day2_min = cells[37].font.text.strip()
    day2_forecast = cells[40].font.text.strip()

    day3_date = cells[41].font.text.strip()
    day3_max = cells[43].font.text.strip()
    day3_min = cells[42].font.text.strip()
    day3_forecast = cells[45].font.text.strip()

    day4_date = cells[46].font.text.strip()
    day4_max = cells[48].font.text.strip()
    day4_min = cells[47].font.text.strip()
    day4_forecast = cells[50].font.text.strip()

    day5_date = cells[51].font.text.strip()
    day5_max = cells[53].font.text.strip()
    day5_min = cells[52].font.text.strip()
    day5_forecast = cells[55].font.text.strip()

    day6_date = cells[56].font.text.strip()
    day6_max = cells[58].font.text.strip()
    day6_min = cells[57].font.text.strip()
    day6_forecast = cells[60].font.text.strip()

    day7_date = cells[61].font.text.strip()
    day7_max = cells[63].font.text.strip()
    day7_min = cells[62].font.text.strip()
    day7_forecast = cells[65].font.text.strip()

    return {
        'temperature': {
            'max': {
                'value': float(max_temp),
                'departure': float(max_dep)
            },
            'min': {
                'value': float(min_temp),
                'departure': float(min_dep)
            }
        },
        'humidity': {
            'morning': float(rh_0830),
            'evening': float(rh_1730)
        },
        'astronomical': {
            'sunrise': sunrise,
            'sunset': sunset,
            'moonrise': moonrise,
            'moonset': moonset
        },
        'forecast': [
            {
                'day': 1,
                'date': day1_date,
                'max': float(day1_max),
                'min': float(day1_min),
                'condition': day1_forecast
            },
            {
                'day': 2,
                'date': day2_date,
                'max': float(day2_max),
                'min': float(day2_min),
                'condition': day2_forecast
            },
            {
                'day': 3,
                'date': day3_date,
                'max': float(day3_max),
                'min': float(day3_min),
                'condition': day3_forecast
            },
            {
                'day': 4,
                'date': day4_date,
                'max': float(day4_max),
                'min': float(day4_min),
                'condition': day4_forecast
            },
            {
                'day': 5,
                'date': day5_date,
                'max': float(day5_max),
                'min': float(day5_min),
                'condition': day5_forecast
            },
            {
                'day': 6,
                'date': day6_date,
                'max': float(day6_max),
                'min': float(day6_min),
                'condition': day6_forecast
            },
            {
                'day': 7,
                'date': day7_date,
                'max': float(day7_max),
                'min': float(day7_min),
                'condition': day7_forecast
            }
        ]
    }


In [8]:
[
    {
        "station": "Shivajinagar",
        "jurisdiction": "Pune",
        "region": "Maharashtra",
        "stationId": 43063
    },
    {
        "station": "Santacruz",
        "jurisdiction": "Mumbai",
        "region": "Maharashtra",
        "stationId": 43003
    },
    {
        "station": "Colaba",
        "jurisdiction": "Mumbai",
        "region": "Maharashtra",
        "stationId": 43057
    },
    {
        "station": "Colaba",
        "jurisdiction": "Mumbai",
        "region": "Maharashtra",
        "stationId": 43057
    },
    {
        "station": "Hyderabad",
        "jurisdiction": "Hyderabad",
        "region": "Telangana",
        "stationId": 43128
    },
    {
        "station": "Bengaluru",
        "jurisdiction": "Bengaluru",
        "region": "Karnataka",
        "stationId": 43295
    },
    {
        "station": "Minambakkam",
        "jurisdiction": "Chennai",
        "region": "Tamil Nadu",
        "stationId": 43279
    },
    {
        "station": "Nungambakkam",
        "jurisdiction": "Chennai",
        "region": "Tamil Nadu",
        "stationId": 43279
    },
    {
        "station": "Visakhapatnam",
        "jurisdiction": "Visakhapatnam",
        "region": "Andhra Pradesh",
        "stationId": 43149
    },
    {
        "station": "Thiruvananthapuram",
        "jurisdiction": "Thiruvananthapuram",
        "region": "Kerala",
        "stationId": 43371
    },
    {
        "station": "Bhubaneswar",
        "jurisdiction": "Bhubaneswar",
        "region": "Odisha",
        "stationId": 42971
    },
    {
        "station": "Gopalpur",
        "jurisdiction": "Gopalpur",
        "region": "Odisha",
        "stationId": 43049
    },
    {
        "station": "Puducherry",
        "jurisdiction": "Puducherry",
        "region": "Puducherry",
        "stationId": 43328
    },
    {
        "station": "Lalpur",
        "jurisdiction": "Raipur",
        "region": "Chhattisgarh",
        "stationId": 42874
    },
    {
        "station": "Arera",
        "jurisdiction": "Bhopal",
        "region": "Madhya Pradesh",
        "stationId": 42667
    },
    {
        "station": "Ahmedabad",
        "jurisdiction": "Ahmedabad",
        "region": "Gujarat",
        "stationId": 42647
    },
    {
        "station": "Indian Institute of Public Health",
        "jurisdiction": "Gandhinagar",
        "region": "Gujarat",
        "stationId": 99959
    },
    {
        "station": "Babasaheb Ambedkar International Airport",
        "jurisdiction": "Nagpur",
        "region": "Maharashtra",
        "stationId": 42867
    },
    {
        "station": "Birsa Munda Airport",
        "jurisdiction": "Ranchi",
        "region": "Jharkhand",
        "stationId": 42701
    },
    {
        "station": "Jaipur International Airport",
        "jurisdiction": "Jaipur",
        "region": "Rajasthan",
        "stationId": 42348
    },
    {
        "station": "Chaudhary Charan Singh International Airport",
        "jurisdiction": "Lucknow",
        "region": "Uttar Pradesh",
        "stationId": 42369
    },
    {
        "station": "Agra Airport",
        "jurisdiction": "Agra",
        "region": "Uttar Pradesh",
        "stationId": 42260
    },
    {
        "station": "Jay Prakash Narayan Airport",
        "jurisdiction": "Patna",
        "region": "Bihar",
        "stationId": 42492
    },
    {
        "station": "Netaji Subhas Chandra Bose International Airport",
        "jurisdiction": "Kolkata",
        "region": "West Bengal",
        "stationId": 42809
    },
    {
        "station": "Safdarjung Airport",
        "jurisdiction": "New Delhi",
        "region": "Delhi",
        "stationId": 42182
    },
    {
        "station": "Indira Gandhi International Airport",
        "jurisdiction": "New Delhi",
        "region": "Delhi",
        "stationId": 42181
    },
    {
        "station": "Gurugram",
        "jurisdiction": "Gurugram",
        "region": "Haryana",
        "stationId": 42178
    },
    {
        "station": "Rohtak",
        "jurisdiction": "Rohtak",
        "region": "Haryana",
        "stationId": 42176
    },
    {
        "station": "Chandigarh",
        "jurisdiction": "Chandigarh",
        "region": "Chandigarh",
        "stationId": 42105
    },
    {
        "station": "Ludhiana",
        "jurisdiction": "Ludhiana",
        "region": "Punjab",
        "stationId": 42099
    },
    {
        "station": "Sri Guru Ram Dass Jee International Airport",
        "jurisdiction": "Amritsar",
        "region": "Punjab",
        "stationId": 42071
    },
    {
        "station": "Jammu",
        "jurisdiction": "Jammu",
        "region": "Jammu and Kashmir",
        "stationId": 42056
    },
    {
        "station": "Srinagar",
        "jurisdiction": "Srinagar",
        "region": "Jammu and Kashmir",
        "stationId": 42027
    },
    {
        "station": "Leh",
        "jurisdiction": "Leh",
        "region": "Ladakh",
        "stationId": 42034
    },
    {
        "station": "Dharamshala",
        "jurisdiction": "Dharamshala",
        "region": "Himachal Pradesh",
        "stationId": 42062
    },
    {
        "station": "Mokhampur",
        "jurisdiction": "Dehradun",
        "region": "Uttarakhand",
        "stationId": 42111
    },
    {
        "station": "Darjeeling",
        "jurisdiction": "Darjeeling",
        "region": "West Bengal",
        "stationId": 42293
    },
    {
        "station": "Gangtok",
        "jurisdiction": "Gangtok",
        "region": "Sikkim",
        "stationId": 42299
    },
    {
        "station": "Guwahati",
        "jurisdiction": "Guwahati",
        "region": "Assam",
        "stationId": 42410
    },
    {
        "station": "Itanagar",
        "jurisdiction": "Itanagar",
        "region": "Arunachal Pradesh",
        "stationId": 42308
    },
    {
        "station": "Kohima",
        "jurisdiction": "Kohima",
        "region": "Nagaland",
        "stationId": 42527
    },
    {
        "station": "Imphal Airport",
        "jurisdiction": "Imphal",
        "region": "Manipur",
        "stationId": 42623
    },
    {
        "station": "Lengpui Airport",
        "jurisdiction": "Aizawl",
        "region": "Mizoram",
        "stationId": 42727
    },
    {
        "station": "Agartala",
        "jurisdiction": "Agartala",
        "region": "Tripura",
        "stationId": 42724
    },
    {
        "station": "Shillong",
        "jurisdiction": "Shillong",
        "region": "Meghalaya",
        "stationId": 42516
    }
]

[{'station': 'Shivajinagar',
  'jurisdiction': 'Pune',
  'region': 'Maharashtra',
  'stationId': 43063},
 {'station': 'Santacruz',
  'jurisdiction': 'Mumbai',
  'region': 'Maharashtra',
  'stationId': 43003},
 {'station': 'Colaba',
  'jurisdiction': 'Mumbai',
  'region': 'Maharashtra',
  'stationId': 43057},
 {'station': 'Colaba',
  'jurisdiction': 'Mumbai',
  'region': 'Maharashtra',
  'stationId': 43057},
 {'station': 'Hyderabad',
  'jurisdiction': 'Hyderabad',
  'region': 'Telangana',
  'stationId': 43128},
 {'station': 'Bengaluru',
  'jurisdiction': 'Bengaluru',
  'region': 'Karnataka',
  'stationId': 43295},
 {'station': 'Minambakkam',
  'jurisdiction': 'Chennai',
  'region': 'Tamil Nadu',
  'stationId': 43279},
 {'station': 'Nungambakkam',
  'jurisdiction': 'Chennai',
  'region': 'Tamil Nadu',
  'stationId': 43279},
 {'station': 'Visakhapatnam',
  'jurisdiction': 'Visakhapatnam',
  'region': 'Andhra Pradesh',
  'stationId': 43149},
 {'station': 'Thiruvananthapuram',
  'jurisdicti